In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras
from PIL import Image
from matplotlib import cm

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename):
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    try:
        assert ds.BodyPartExamined == 'CHEST' and ds.PatientPosition in ['PA','AP'] and ds.Modality == 'DX'
    except AssertionError as ex:
        print('Invalid DICOM')
    img = ds.pixel_array
    return img

# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size):
    normalized_img = img / 255
    img_dim0 = img_size[0]
    img_dim1 = img_size[1]
    img_dim2 = img_size[2]
    img_dim3 = img_size[3]
    
    img = Image.fromarray(img)
    resized_img = img.resize((img_dim1, img_dim2))
    resized_img = np.asarray(resized_img)
    
    width, height, channels = img_dim1, img_dim2, 1
    proc_img = np.tile(resized_img.reshape((img_dim0, width, height, channels)), img_dim3)
    proc_img = (proc_img - img_mean) / img_std
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    file_path = open(model_path, 'r')
    model = keras.models.model_from_json(file_path.read())
    model.load_weights(weight_path)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh):
    prediction = 'Pneumonia' if model.predict(img) > thresh else 'Non-Pneumonia'
    return prediction

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json' # path to saved model
weight_path = '{}_my_model.best.hdf5'.format('xray_class') # path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) # loads model
thresh = 0.50 # loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
    
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Non-Pneumonia
Load file test2.dcm ...
Non-Pneumonia
Load file test3.dcm ...
Pneumonia
Load file test4.dcm ...
Invalid DICOM
Non-Pneumonia
Load file test5.dcm ...
Invalid DICOM
Non-Pneumonia
Load file test6.dcm ...
Invalid DICOM
Non-Pneumonia
